In [1]:
import sys

sys.path.append("..")

In [2]:
import pandas as pd

from utils.timer import timer
from utils.excels import each_row, save_excel

In [3]:
import os

Bytes_Per_Sec = 201953523 / 333.0

@timer
def read_excel(file, name=None, rows=None):
    """
    读取Excel
    
    默认读取所有sheet, 可以指定
    默认读取所有行, 可以指定: 指定小批量行数并不会节约时间
    读取200MB约350s
    """
    file_bytes = os.path.getsize(file)
    print("predict cost: %.5fs" % (file_bytes / Bytes_Per_Sec))
    return pd.read_excel(file, nrows=rows, sheet_name=name)


In [7]:
abs_path = 'D:\\Documents\\vivo\\'


## 用户处理

In [8]:
user_file='VIVO抖音项目-用户信息-0606.xlsx'

user_sheet = read_excel(abs_path + user_file, "模板")
print(user_sheet.shape)

# start at 1559920349.49411
predict cost: 0.65696s
# end at 1559920350.01692
total cost: 0.52280
(1389, 29)


In [9]:
user_tags = {}

def user_parser(idx, key, row):
    user_tag = {}
    
    uid = row["uid"]
    name = row["user_name"]
    nick = row["register_id"]
    fans = row["fans_num"]
    
    user_tag["uid"] = uid
    user_tag["name"] = name
    user_tag["fans"] = fans
    user_tag["register_id"] = nick
    
    user_tags[uid] = user_tag
    
    return True
    
each_row(user_sheet, user_parser, 1000)

# start at 1559920359.64146
current time: 1559920359.776255
current idx: 1000
# end at 1559920359.80622
total cost: 0.16476


In [10]:
print(len(user_tags))
for k, v in user_tags.items():
    print(k, v)
    break

1389
100003454600 {'uid': 100003454600, 'name': '极好金服', 'fans': 288797, 'register_id': 'jihaojinfu'}


## 视频处理

In [11]:
video_file = 'analysis_task_20190606230755_761_93_video.xlsx'

video_sheet = read_excel(abs_path + video_file, "sheet0")
print(video_sheet.shape)

# start at 1559920366.11897
predict cost: 0.63992s
# end at 1559920366.45012
total cost: 0.33115
(760, 30)


In [12]:
brand_file = "VIVO品牌码表.xlsx"

brand_sheet = read_excel(abs_path + brand_file, "Sheet1")
print(brand_sheet.shape)

# start at 1559920370.49684
predict cost: 0.02142s
# end at 1559920370.53473
total cost: 0.03790
(56, 5)


In [13]:
brandKB = {}

def brand_parser(idx, key, row):
    brand_name = row['品牌'].strip()
    if brand_name not in brandKB:
        brandKB[brand_name] = set()
    
    if type(row['品牌产品']) is str:
        brand_type_name = row['品牌产品'].strip()
        if brand_type_name:
            brandKB[brand_name].add(brand_type_name)
    
    brand_keyword_str = row['关键词'].strip()
    if brand_keyword_str.startswith('#'):
        brandKB[brand_name].add(brand_keyword_str)
    else:
        for kw in brand_keyword_str.split(','):
            if type(kw) is str:
                brandKB[brand_name].add(kw)
    
    return True
                
each_row(brand_sheet, brand_parser)
print(len(brandKB))


# start at 1559920372.35246
# end at 1559920372.35845
total cost: 0.00598
6


In [14]:
def kw_match(content, keywords):
    """
    关键词匹配
    """
    for kw in keywords:
        if kw in content:
            return True
    return False

def brand_match(content, kb):
    """
    品牌匹配
    """
    for key, value in kb.items():
        if kw_match(content, value):
            return key
    
    return None

# TEST CASE

print(brand_match("vivo", brandKB))
print(brand_match("#我才是实力自拍王", brandKB))
print(brand_match("vivo", brandKB))
print(brand_match("#华为", brandKB))

VIVO
华为
VIVO
None


In [15]:
import time

def common_time_parse(origin):
    """
    时间处理
    """
    real_time = time.localtime() # 默认当前时间
    if type(origin) is str and len(origin) == 19:
        real_time = time.strptime(origin, "%Y-%m-%d %H:%M:%S")
    elif len(str(origin)) == 14:
        real_time = time.strptime(str(origin), "%Y%m%d%H%M%S")

    return real_time

# time.strftime(real_time, "%Y-%m-%d %H:%M:%S")

In [16]:
video_tags = {}

def video_parser(idx, key, row):
    video_tag = {}
    
    url = row['url']
    vid = url.split('/')[5]
    uid = row["user_item_id"]
    
    publish_date = time.strftime("%Y-%m-%d %H:%M:%S", common_time_parse(row["publish_date"]))
    title = row["title"]
    content = row['content']
    brand = ""
    if type(content) is str:
        brand = brand_match(content, brandKB)
        
    video_tag["vid"] = vid
    video_tag["url"] = url
    video_tag["publish_date"] = publish_date
    video_tag["title"] = title
    video_tag["content"] = content
    video_tag["brand"] = brand
       
    user_tag = {}
    if uid in user_tags:
        user_tag = user_tags[uid]
    else:
        print("user not exist: %s" % uid)
    for k, v in user_tag.items():
        video_tag["user_" + k] = v

    video_tags[vid] = video_tag
    return True
    
each_row(video_sheet, video_parser, 1000)
print(len(video_tags))

# start at 1559920382.51724
user not exist: 4173365255879406
# end at 1559920382.72479
total cost: 0.20755
760


In [17]:
print(len(video_tags))
for k, v in video_tags.items():
    print(k)
    print(v)
    break

760
6641823319651781896
{'vid': '6641823319651781896', 'url': 'https://www.iesdouyin.com/share/video/6641823319651781896/?region=CN&mid=6641823330779269902&u_code=jdajj3e5&titleType=title', 'publish_date': '2019-01-02 17:06:39', 'title': '@海飞数码创作的原声', 'content': '目前拍照最好的十部手机，让我们大喊一句“国产牛逼”#手机 #华为 #欢脱喜提开门红', 'brand': None, 'user_uid': 74968768187, 'user_name': '海飞数码', 'user_fans': 1308577, 'user_register_id': '118900302'}


## 评论处理



In [18]:
comment_file = 'analysis_task_20190606230755_761_93.xlsx'
comment_all_sheet = read_excel(abs_path + comment_file)
print(len(comment_all_sheet))

# start at 1559920397.34344
predict cost: 333.00000s
# end at 1559920791.18296
total cost: 393.83952
3


> 安全隔离带

In [28]:
from elasticsearch import Elasticsearch

from utils.elasticsearchs import pretty_print, create_index, insert_bulk

In [29]:
proj_es_17 = Elasticsearch([{'host':'proj1', 'port':9200}])

1. 准备索引

In [30]:
fields = [
    "item_id|long",
    "user_item_id|long",
    "video_item_id|long",
    "keyword|string|words",
    "module|string|not_analyzed",
    "site|string|not_analyzed",
    "author|string|words",
    "title|string|words",
    "content|string|words",
    "like_count|long",
    "url|string|not_analyzed",
    "replies|long",
    "data_type|string|not_analyzed",
    "publish_date|long",
    "update_date|long",
    "sourceCrawlerId|long",
    "video_url|string|not_analyzed",
    "video_brand|string|not_analyzed",
    "video_content|string|words",
    "video_publish_date|long",
    "video_user_fans|long",
    "video_user_name|string|words",
    "video_user_register_id|string|not_analyzed"
]

index = "vivo_douyin_v0"
doc_type = "douyin"

In [31]:
create_res = create_index(proj_es_17, index, doc_type, fields)
print(create_res)

index body: {
  "mappings": {
    "douyin": {
      "dynamic": "strict",
      "properties": {
        "author": {
          "analyzer": "words",
          "type": "string"
        },
        "content": {
          "analyzer": "words",
          "type": "string"
        },
        "data_type": {
          "doc_values": true,
          "index": "not_analyzed",
          "type": "string"
        },
        "item_id": {
          "type": "long"
        },
        "keyword": {
          "analyzer": "words",
          "type": "string"
        },
        "like_count": {
          "type": "long"
        },
        "module": {
          "doc_values": true,
          "index": "not_analyzed",
          "type": "string"
        },
        "publish_date": {
          "type": "long"
        },
        "replies": {
          "type": "long"
        },
        "site": {
          "doc_values": true,
          "index": "not_analyzed",
          "type": "string"
        },
        "sourceCrawlerId": {
 

2. 写入数据

In [75]:
def flush(es, index, doc_type, data, size=3000, force=False):
    if not force:
        if len(data) < size:
            return False
        
    origin_size = len(data)
    success, info = insert_bulk(es, index, doc_type, data)
    data.clear()
    
    if success < origin_size:
        print(info)
        
    return True


In [76]:
current_sheet = "sheet0"

In [79]:
import math

comment_test_sheet = comment_all_sheet[current_sheet]
print(comment_test_sheet.shape)

comment_datas = {}
def comment_parser(idx, key, row):
    comment_data = {}

    # 1. origin info
    cid = row["item_id"]
    url = row["url"]
    vid = url.split('/')[5]

    comment_data["item_id"] = cid
    comment_data["url"] = url
    comment_data["video_item_id"] = vid
    comment_data["publish_date"] = int(time.mktime(common_time_parse(row["publish_date"])))
    comment_data["update_date"] = int(time.mktime(common_time_parse(row["update_date"])))
 
    same_keys = set(["user_item_id", 
                      "keyword", "module", "site",
                      "author", "title", "content",
                      "like_count", "replies", "data_type",
                      "sourceCrawlerId"])
    for k in same_keys:
        if type(row[k]) is float and math.isnan(row[k]):
            continue
        comment_data[k] = row[k]

    # 2. video info
    video_tag = video_tags[vid]
    comment_data["video_publish_date"] = int(time.mktime(common_time_parse(video_tag["publish_date"])))

    video_keys = set(["url", "brand", "content",
                     "user_fans", "user_name", "user_register_id"])
    for k in video_keys:
        if k in video_tag: # 有一个用户信息没抓到
            comment_data["video_" + k] = video_tag[k]

    # 3. store
    comment_datas[cid] = comment_data
    flush(proj_es_17, index, doc_type, comment_datas)
#     print("after insert: %d" % len(comment_datas))

    return True
    
each_row(comment_test_sheet, comment_parser, 5000)

# print("before %d" % len(comment_datas))
flush(proj_es_17, index, doc_type, comment_datas, force=True)
comment_datas.clear()
# print("after %d" % len(comment_datas))

# chunksize=4, csv才有

(500000, 23)
# start at 1559935577.53301
action size: 3000
current time: 1559935581.710617
current idx: 5000
action size: 3000
action size: 3000
current time: 1559935587.134010
current idx: 10000
action size: 3000
action size: 3000
current time: 1559935593.875381
current idx: 15000
action size: 3000
current time: 1559935597.256660
current idx: 20000
action size: 3000
action size: 3000
current time: 1559935602.292340
current idx: 25000
action size: 3000
action size: 3000
current time: 1559935607.051809
current idx: 30000
action size: 3000
current time: 1559935610.799246
current idx: 35000
action size: 3000
action size: 3000
current time: 1559935616.024639
current idx: 40000
action size: 3000
action size: 3000
current time: 1559935620.887657
current idx: 45000
action size: 3000
current time: 1559935624.337826
current idx: 50000
action size: 3000
action size: 3000
current time: 1559935629.740063
current idx: 55000
action size: 3000
action size: 3000
current time: 1559935636.125715
current

In [80]:
# print(len(comment_tags))
# for k, v in comment_tags.items():
#     print(k)
#     print(v)
#     break
# print(len(comment_infos))
# for k, v in comment_infos.itmes():
#     print(k)
#     print(v)
#     break

In [81]:
# comment_infos_df = pd.DataFrame(comment_infos)
# comment_tags_df = pd.DataFrame(comment_tags)

# comment_df = pd.concat([comment_infos_df, comment_tags_df], axis=0)

# print(comment_infos_df.shape)
# print(comment_tags_df.shape)
# print(comment_df.shape)

In [82]:
# save_excel(comment_df.T, abs_path + "comment-" + current_sheet + ".xlsx")